# 1. ETL Pipeline for Pre-Processing the Files

#### Importing Python packages 

In [1]:
# importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))

# uncomment the code below to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# checking the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# 2. Building Apache Cassandra tables 

Working with the CSV file titled <font color=red>event_datafile_new.csv</font>. The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Creating a connection to a Cassandra instance 
# (127.0.0.1) if you have a locally installed Apache Cassandra instance 

from cassandra.cluster import Cluster
cluster = Cluster()

# Creating session to establish connection and execute queries 
session = cluster.connect()

#### Creating Keyspace

In [6]:
# Creating a keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [7]:
# Connecting to the keyspace

try:
    session.set_keyspace('music')
except Exception as e:
    print(e)

### Creating queries & modelling the database tables on these queries 


##### Query 1. Getting artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

* `SELECT artist, song, length FROM artist_library WHERE sessionId = 338 AND itemInSession = 4`


##### Query 2. Getting name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
* `SELECT artist, song, firstName, lastName FROM artist_user_library WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession`


##### Query 3. Getting user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

* `SELECT firstName, lastName FROM user_library WHERE song = 'All Hands Against His Own'`


#### Query 1

In [8]:
# Query 1: `SELECT artist, song, length FROM artist_library WHERE sessionId = 338 AND itemInSession = 4`
# The primary key should combine SessionId as the partitioning column and ItemInSession as a clustering column.

query = "CREATE TABLE IF NOT EXISTS song_by_session_by_item "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                  

In [9]:
# Inserting data in the table 

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_session_by_item (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Assigning column elements for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
# Validating the data model by verifying that the data has been inserted into the table

query = "SELECT artist, song, length FROM song_by_session_by_item WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query 2

In [11]:
# Query 2: `SELECT artist, song, firstName, lastName FROM artist_user_library WHERE userId = 10 AND sessionId = 182`
# The primary key should combine userId as the partitioning column and sessionId a clustering column. itemInSession should be a clustering column. 

query = "CREATE TABLE IF NOT EXISTS artist_song_username_by_user_by_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                  
                    

In [12]:
# Inserting data in the table 
        
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_username_by_user_by_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assigning column elements for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# Validating the data model by verifying that the data has been inserted into the table

query = "SELECT artist, song, firstName, lastName FROM artist_song_username_by_user_by_session WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3

In [14]:
# Query 3: `SELECT firstName, lastName FROM user_library WHERE song = 'All Hands Against His Own'`
# The primary key should have song as the partitioning column and userId as the clustering column

query = "CREATE TABLE IF NOT EXISTS username_by_song "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)                        

In [15]:
# Inserting data in the table 
        
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO username_by_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Assigning column elements for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
# Validating the data model by verifying that the data has been inserted into the table

query = "SELECT userId, firstName, lastName FROM username_by_song WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userid, row.firstname, row.lastname)

29 Jacqueline Lynch
80 Tegan Levine
95 Sara Johnson


### Dropping tables

In [17]:
# Dropping the table before closing the sessions

query = "drop table song_by_session_by_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table artist_song_username_by_user_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table username_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()